In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM_LBLNEW['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
DS_FLUX_WGT.sel(igg=1).sel(g=1).to_dataframe();

In [14]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    
    


In [15]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    s_outputs_byg = 'Outputs by g-interval'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_outputs_byg))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [16]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 1013f91
molecule = {'co2': 0.0004}
band = [4]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a22ab94
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 4]
  nv = 1000
  option_compute_btable = 1
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66, 1.66, 1.66)]
  wgt = [(0, 0.5, 0.4, 0.6, 0.65), (0.7, 0.7, 0.55, 0.6)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`                 
            layer     coolrg                    i layer     coolrg
pressure                                                          
0.000312        1  76.911573                    1     1  54.636205
0.000750        2  41.020532                    1     2  41.629703
0.001052        3  33.259869                    1     3  36.060282
0.001476        4  26.397425                    1     4  29.531372
0.002070        5  20.664232                    1     5  22.680055
0.002904        6  16.081310                    1     6  16.382545
0.004074        7  12.530285                    1     7  11.549865
0.005714        8   9.827018                    1     8   8.492917
0.008015        9   7.770893                    1     9   6.801298
0.011243       10   6.180803                    1    10   5.730593
0.015771       11   4.903649                    1    11   4.712805
0.022122       12   3.810097                    1    12   3.608425
0.031031       13   2.745222                    1    13   2.392883
0.043528       14   1.399978                    1    14   0.992763
0.061057       15   0.979141                    1    15   0.634240
0.085645       16   1.752999                    1    16   1.560913
0.120136       17   2.555489                    1    17   2.559920
0.168516       18   3.176324                    1    18   3.352582
0.236378       19   3.831257                    1    19   4.046337
0.331549       20   4.628079                    1    20   4.670991
0.465100       21   5.780301                    1    21   5.568341
0.652400       22   7.472781                    1    22   7.159129
0.915100       23   7.603271                    1    23   7.459041
1.283650       24   6.451719                    1    24   6.408524
1.800600       25   5.474882                    1    25   5.300224
2.525700       26   4.427503                    1    26   4.162870
3.542800       27   3.636540                    1    27   3.592638
4.969550       28   3.059310                    1    28   3.192447
6.970850       29   2.591584                    1    29   2.693923
9.778100       30   2.179984                    1    30   2.197782
13.715850      31   1.799270                    1    31   1.826057
19.239350      32   1.443215                    1    32   1.415439
26.987250      33   1.121665                    1    33   1.091365
37.855300      34   0.874324                    1    34   0.933570
53.100050      35   0.507275                    1    35   0.568150
73.887500      36   0.034923                    1    36   0.021046
97.662500      37  -0.289156                    1    37  -0.323061
121.437500     38  -0.219236                    1    38  -0.220491
145.212500     39   0.020826                    1    39   0.036257
168.987500     40   0.075532                    1    40   0.082616
192.762500     41   0.099899                    1    41   0.102419
216.537500     42   0.111780                    1    42   0.112123
240.312500     43   0.116942                    1    43   0.114397
264.087500     44   0.118045                    1    44   0.111346
287.862500     45   0.116553                    1    45   0.105641
311.637500     46   0.113290                    1    46   0.099258
335.412500     47   0.108849                    1    47   0.092935
359.187500     48   0.103653                    1    48   0.088446
382.962500     49   0.097977                    1    49   0.084017
406.737500     50   0.092141                    1    50   0.080868
430.512500     51   0.086304                    1    51   0.078446
454.287500     52   0.080623                    1    52   0.074806
478.062500     53   0.075138                    1    53   0.072659
501.837500     54   0.069950                    1    54   0.068529
525.612500     55   0.065174                    1    55   0.066545
549.387500     56   0.060815                    1    56   0.062177
573.162500     57   0.057038                    1    57   0.0

# Fluxes by Layer

CRD                       CLIRAD             \
                        flug       fldg      fnetg      i       flug   
pressure    level                                                      
0.000000    1     -16.254068   0.000000 -16.254068      1 -15.835989   
0.000624    2     -16.254436   0.006056 -16.248379      1 -15.836385   
0.000876    3     -16.254603   0.007445 -16.247157      1 -15.836551   
0.001229    4     -16.254812   0.009044 -16.245768      1 -15.836764   
0.001723    5     -16.255074   0.010854 -16.244221      1 -15.837038   
0.002417    6     -16.255405   0.012883 -16.242522      1 -15.837389   
0.003391    7     -16.255824   0.015157 -16.240667      1 -15.837845   
0.004757    8     -16.256362   0.017721 -16.238640      1 -15.838437   
0.006672    9     -16.257057   0.020647 -16.236411      1 -15.839212   
0.009359    10    -16.257970   0.024033 -16.233937      1 -15.840240   
0.013128    11    -16.259189   0.028011 -16.231178      1 -15.841615   
0.018415    12    -16.260851   0.032745 -16.228107      1 -15.843498   
0.025830    13    -16.263185   0.038425 -16.224760      1 -15.846128   
0.036232    14    -16.266652   0.045275 -16.221377      1 -15.849993   
0.050823    15    -16.272641   0.053684 -16.218958      1 -15.856426   
0.071291    16    -16.282331   0.065747 -16.216584      1 -15.866551   
0.100000    17    -16.295514   0.084892 -16.210622      1 -15.880189   
0.140271    18    -16.312196   0.113765 -16.198431      1 -15.897298   
0.196760    19    -16.332902   0.155726 -16.177177      1 -15.918265   
0.275997    20    -16.357731   0.216515 -16.141216      1 -15.942997   
0.387100    21    -16.385269   0.304963 -16.080306      1 -15.969916   
0.543100    22    -16.408114   0.434624 -15.973490      1 -15.991791   
0.761700    23    -16.399970   0.619986 -15.779984      1 -15.983624   
1.068500    24    -16.348487   0.844825 -15.503661      1 -15.933632   
1.498800    25    -16.263914   1.089110 -15.174804      1 -15.851964   
2.102400    26    -16.142986   1.359641 -14.783346      1 -15.734984   
2.949000    27    -15.993169   1.653839 -14.339330      1 -15.589418   
4.136600    28    -15.814574   1.986831 -13.827743      1 -15.413519   
5.802500    29    -15.601602   2.377576 -13.224026      1 -15.202464   
8.139200    30    -15.348183   2.841504 -12.506679      1 -14.947989   
11.417000   31    -15.048074   3.387837 -11.660237      1 -14.643612   
16.014700   32    -14.695350   4.015049 -10.680300      1 -14.282159   
22.464000   33    -14.284245   4.706511  -9.577734      1 -13.860863   
31.510500   34    -13.800127   5.424394  -8.375733      1 -13.363249   
44.200100   35    -13.154004   6.092532  -7.061472      1 -12.700834   
62.000000   36    -12.436785   6.444915  -5.991870      1 -11.969161   
85.775000   37    -12.199267   6.305751  -5.893515      1 -11.725776   
109.550000  38    -12.862807   6.154936  -6.707871      1 -12.430968   
133.325000  39    -14.076385   6.751076  -7.325309      1 -13.710903   
157.100000  40    -15.303670   8.037015  -7.266656      1 -14.992942   
180.875000  41    -16.528801   9.474868  -7.053934      1 -16.262217   
204.650000  42    -17.745588  10.973001  -6.772587      1 -17.512914   
228.425000  43    -18.950082  12.492304  -6.457778      1 -18.746532   
252.200000  44    -20.139331  14.010899  -6.128432      1 -19.959159   
275.975000  45    -21.311691  15.515712  -5.795979      1 -21.151374   
299.750000  46    -22.466145  16.998415  -5.467729      1 -22.323407   
323.525000  47    -23.602153  18.453483  -5.148670      1 -23.472351   
347.300000  48    -24.719490  19.877374  -4.842116      1 -24.602256   
371.075000  49    -25.818176  21.267980  -4.550197      1 -25.711567   
394.850000  50    -26.898377  22.624115  -4.274262      1 -26.800502   
418.625000  51    -27.960199  23.945436  -4.014763      1 -27.869551   
442.400000  52    -29.003979  25.232275  -3.771704      1 -28.919612   
466.175000  53    -30.030017  26.485374  -3.544643      1 -29.952

In [17]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')